# 0. Разобраться с minimap2

```
$ minimap2 -ax map-pb data/reference.fasta data/pacbio_40x.fq.gz > pacbio.sam
[M::mm_idx_gen::0.151*0.90] collected minimizers
[M::mm_idx_gen::0.176*1.17] sorted minimizers
[M::main::0.176*1.17] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.192*1.16] mid_occ = 11
[M::mm_idx_stat] kmer size: 19; skip: 10; is_hpc: 1; #seq: 1
[M::mm_idx_stat::0.201*1.15] distinct minimizers: 606828 (98.95% are singletons); average occurrences: 1.024; average spacing: 7.468
[M::worker_pipeline::58.550*2.79] mapped 41447 sequences
[M::main] Version: 2.16-r922
[M::main] CMD: minimap2 -ax map-pb data/reference.fasta data/pacbio_40x.fq.gz
[M::main] Real time: 58.571 sec; CPU: 163.574 sec; Peak RSS: 1.603 GB

$ minimap2 -ax map-ont data/reference.fasta data/ONT_2D.fastq.gz > ont.sam
[M::mm_idx_gen::0.160*1.02] collected minimizers
[M::mm_idx_gen::0.200*1.40] sorted minimizers
[M::main::0.200*1.40] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::0.218*1.36] mid_occ = 11
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::0.231*1.34] distinct minimizers: 838533 (98.18% are singletons); average occurrences: 1.034; average spacing: 5.352
[M::worker_pipeline::22.272*2.50] mapped 30810 sequences
[M::main] Version: 2.16-r922
[M::main] CMD: minimap2 -ax map-ont data/reference.fasta data/ONT_2D.fastq.gz
[M::main] Real time: 22.287 sec; CPU: 55.640 sec; Peak RSS: 0.917 GB

$ samtools flagstat ont.sam
32103 + 0 in total (QC-passed reads + QC-failed reads)
1072 + 0 secondary
221 + 0 supplementary
0 + 0 duplicates
29565 + 0 mapped (92.09% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)

$ samtools flagstat pacbio.sam
52081 + 0 in total (QC-passed reads + QC-failed reads)
1736 + 0 secondary
8898 + 0 supplementary
0 + 0 duplicates
51199 + 0 mapped (98.31% : N/A)
0 + 0 paired in sequencing
0 + 0 read1
0 + 0 read2
0 + 0 properly paired (N/A : N/A)
0 + 0 with itself and mate mapped
0 + 0 singletons (N/A : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)
```

# 1. Статистики ридов PacBio и Oxford Nanopores.
## Покрытие

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os
import pysam
from itertools import zip_longest, islice
import pysamstats
from Bio import SeqIO
import gzip

def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

In [3]:
coverages = []
samfile = pysam.AlignmentFile("work/pacbio.sam", "rb")
samfilelength = samfile.lengths[0]
nonzerobp = 0
tlen = []
for pileupcolumn in grouper(samfile.pileup(), 10000):
    bucket = [x.nsegments for x in pileupcolumn if x != None]
    nonzerobp += np.count_nonzero(bucket)
    avg_coverage = np.mean(bucket)
    coverages.append(avg_coverage)
samfile.close()

plt.bar(list(range(0, len(coverages))), coverages)
plt.title("Pacbio")
plt.ylabel("Coverage")
plt.xlabel("Ten thousands of bp")
print(f"Avg coverage {np.mean(coverages)}")
print(f"Genome covered {nonzerobp / samfilelength * 100}%")

NotImplementedError: pileup of samfiles not implemented yet

In [ ]:
coverages = []
samfile = pysam.AlignmentFile("work/ont.sam", "rb")
samfilelength = samfile.lengths[0]
nonzerobp = 0
tlen = []
for pileupcolumn in grouper(samfile.pileup(), 10000):
    bucket = [x.nsegments for x in pileupcolumn if x != None]
    nonzerobp += np.count_nonzero(bucket)
    avg_coverage = np.mean(bucket)
    coverages.append(avg_coverage)
samfile.close()

plt.bar(list(range(0, len(coverages))), coverages)
plt.title("ONT")
plt.ylabel("Coverage")
plt.xlabel("Ten thousands of bp")
print(f"Avg coverage {np.mean(coverages)}")
print(f"Genome covered {nonzerobp / samfilelength * 100}%")

## Процент ошибок


## Распределение типов замен


## Распределение длин инделов
